In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection  import train_test_split
from sklearn.metrics  import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

url = "https://loterias.caixa.gov.br/Paginas/Lotofacil.aspx"


In [2]:
def open():
    driver = webdriver.Chrome()
    # Acessar o site
    driver.get(url)
    return driver 


# Recuperar a dezena necessária 
def recuperarDezenas(driver):
    # Encontrar a div que contém as dezenas sorteadas
    div_dezenas = driver.find_element(By.CLASS_NAME, 'resultado-loteria')

    # Encontrar todas as tags <li> que contêm as dezenas
    dezenas_tags = div_dezenas.find_elements(By.TAG_NAME, 'li')

    # Extrair e imprimir as dezenas sorteadas
    return [dezena.text.strip() for dezena in dezenas_tags]

def close(driver):
    # Fechar o navegador
    driver.quit()
    
def recuperarData(driver):
    # Encontrar o elemento usando XPath (vamos usar o título da página como exemplo)
    xpath = "//*[@id='wp_resultados']/div[1]/div/h2/span"
    elemento = driver.find_element("xpath",xpath) 
    # Extrair e imprimir o texto
    data = elemento.text
    data_limp = data.split()
    data = data_limp[2].replace("(","").replace(")","")
    dia,mes,ano,concurso  = data[:2], data[3:5],data[6:10],data_limp[1]
    
    return  [data,dia,mes,ano,concurso]

def clicar(driver):
    xpath = "//*[@id='wp_resultados']/div[1]/div/div[2]/ul/li[2]/a"
    elemento = driver.find_element("xpath",xpath)
    elemento.click()

def escrever(concurso, driver):
    xpath = "//*[@id='buscaConcurso']"
    elemento = driver.find_element("xpath", xpath)
    elemento.send_keys(concurso)
    elemento.send_keys(Keys.RETURN)
    elemento.clear()

     
   

In [3]:
driver = open()
sorteios = []
for i in range(2586):
    try:
        data, dia, mes, ano, concurso = recuperarData(driver)
        dezenas = recuperarDezenas(driver)
        data, dia, mes, ano, concurso = recuperarData(driver)
        dezenas = recuperarDezenas(driver)
        dezenas.append(data)
        dezenas.append(dia)
        dezenas.append(mes)
        dezenas.append(ano)
        dezenas.append(concurso)
        sorteios.append(dezenas)
        clicar(driver)
    except:
        close(driver)
        time.sleep(3)
        driver = open()
        escrever(concurso,driver)
    time.sleep(3)


close(driver)

In [4]:
df_dezenas = pd.DataFrame(sorteios, columns=['Dezena_1', 'Dezena_2', 'Dezena_3', 'Dezena_4', 'Dezena_5',
                                                   'Dezena_6', 'Dezena_7', 'Dezena_8', 'Dezena_9', 'Dezena_10',
                                                   'Dezena_11', 'Dezena_12', 'Dezena_13', 'Dezena_14', 'Dezena_15',
                                                   'Data_Sorteio','Dia','Mes','Ano','concurso'])

In [5]:
# Machine learning
# 1. Preparação dos Dados
df_dezenas['Data_Sorteio'] = pd.to_datetime(df_dezenas['Data_Sorteio'], format='%d/%m/%Y')


# Especificar o caminho para o arquivo CSV
caminho_arquivo_csv = 'sorteio.csv'

# Salvar o DataFrame em um arquivo CSV separado por ponto e vírgula
df_dezenas.to_csv(caminho_arquivo_csv, sep=';', index=False)


# 2. Criação da Coluna 'Dias_Desde_Inicio'
primeira_data = df_dezenas['Data_Sorteio'].min()
df_dezenas['Dias_Desde_Inicio'] = (df_dezenas['Data_Sorteio'] - primeira_data).dt.days
copy = df_dezenas  
df_dezenas = df_dezenas.drop(['Data_Sorteio','Dia','Mes','Ano'], axis=1)

In [ ]:
# 3. Divisão dos Dados
X = df_dezenas['Dias_Desde_Inicio'].values.reshape(-1, 1)
y = df_dezenas[['Dezena_1', 'Dezena_2', 'Dezena_3', 'Dezena_4', 'Dezena_5',
                'Dezena_6', 'Dezena_7', 'Dezena_8', 'Dezena_9', 'Dezena_10',
                'Dezena_11', 'Dezena_12', 'Dezena_13', 'Dezena_14', 'Dezena_15']]
proximo_dia = df_dezenas['Dias_Desde_Inicio'].max() + 1

In [ ]:
# 4. Escolha do Modelo
modelo1 = LinearRegression()

# 5. Treinamento do Modelo
modelo1.fit(X, y)

# 6. Previsões para a Primeira Data
previsao_primeira_data = modelo1.predict([[proximo_dia]])

prev  = [int(i) for i in previsao_primeira_data[0]]

print(f'Previsão para a primeira data de sorteio: {prev}')

In [ ]:
def numero_mais_frequente_por_coluna(dataset):
    mais_frequentes = {}

    for coluna in dataset.columns:
        # Encontrar o número mais frequente usando o método value_counts
        mais_frequente = dataset[coluna].value_counts().idxmax()
        
        # Armazenar o resultado no dicionário mais_frequentes
        mais_frequentes[coluna] = mais_frequente

    return mais_frequentes

In [ ]:
remove = ['Dias_Desde_Inicio', 'contagem_repeticoes']
resultados = numero_mais_frequente_por_coluna(df_dezenas)
x = [i for k, i in resultados.items() if k not in remove]
print(x)

In [ ]:
df_dezenas.drop(['Dias_Desde_Inicio','contagem_repeticoes'], axis=1)
linhas_duplicadas = df_dezenas.duplicated()
# Adiciona uma coluna contendo a contagem de cada linha
df_dezenas['contagem_repeticoes'] = df_dezenas.groupby(list(df_dezenas.columns)).cumcount() + 1
# Filtra as linhas que se repetem
linhas_repetidas = df_dezenas[linhas_duplicadas]
# Encontra a maior contagem
maior_contagem = linhas_repetidas['contagem_repeticoes'].max()
linhas = linhas_repetidas[linhas_repetidas['contagem_repeticoes'] == maior_contagem]
linhas 

In [ ]:
# Separar features (X) e target (y)
X = dataset.drop("dezenas_sorteio", axis=1)  # Substitua "dezenas_sorteio" pelo nome da coluna alvo
y = dataset["dezenas_sorteio"]

# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
newDataSet = df_dezenas
#for i in range(1,26):
#    newDataSet["numero{}".format(i)] = i 
#newDataSet['dezenas_sorteio'] = newDataSet.apply(lambda row: ''.join(map(str, row[1:16])), axis=1)

In [ ]:
newDataSet